#### Importing packages


In [ ]:
# ! pip install checkscore
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder, DepositTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from checkscore.repeater import retry
import requests
import csv
import concurrent.futures
import json
import pickle

import csv
import os
import time
from pprint import pprint
EXA = 10**18


#### Methods for exa calculations

In [ ]:
EXA = 10**18
halfEXA = EXA // 2
SECONDS_PER_YEAR = 31536000


def exaMul(a: int, b: int) -> int:
    return (halfEXA + (a*b)) // EXA

def exaDiv(a: int, b: int) -> int:
    halfB = b // 2
    return (halfB + (a * EXA)) // b

def exaPow(x: int, n: int) -> int:
    if n % 2 != 0:
        z = x
    else:
        z = EXA

    n = n // 2
    while n != 0:
        x = exaMul(x, x)

        if (n % 2 != 0):
            z = exaMul(z, x)

        n = n // 2
        
    return z



def convertToExa(_amount:int,_decimals:int)-> int:
    if _decimals >= 0:
        return _amount * EXA // (10 ** _decimals)

def convertExaToOther(_amount:int,_decimals:int)->int:
    if _decimals >= 0:
        return _amount * (10 ** _decimals) // EXA



#### Connection Setup

In [ ]:
connections = {
    "mainnet": {"iconservice": "https://ctz.solidwallet.io",
                "nid": 1,
                },
    "sejong":  {"iconservice": "https://sejong.net.solidwallet.io", 
                "nid": 83,
                }
    }


### Set the right addresses and variables here

In [ ]:
# set the required addresses for the SCOREs here
contracts = {
  "mainnet":{"rewardDistribution": "cx4f2d730ad969f5c839229de42184c5e47aefef6f",
              "ommToken": "cx1a29259a59f463a67bb2ef84398b30ca56b5830a",
              "stakedLp":"cx015c7f8884d43519aa2bcf634140bd7328730cb6",
              "rewardWeightController":""
              },
  "sejong":{"rewardDistribution": "cx0e0e5c7bdf60ed110d2db918cbf6941b8eb124eb",
            "ommToken": "cx01c67340ca7bb5e8d1a2bddbfdce40d4addbf7f0",
            "stakedLp":"cx3a64607a523d7c56cc85861b89bce271e17c6e81",
            "rewardWeightController":"cxc79754e827b7dc0a12b58092eae5c5d5ad6f8ce4"
            }}
lps_list ={"sejong":{"OMM/sICX": "cxf31d5a3077dfa9b94c7385e17ec747e8809cc53f",
                     "OMM/USDS": "cxa7c9f0afe73e541aee7030a931aec41b81819e74",
                     "OMM/IUSDC": "cxbdfea4b4d291c3e01ed8548aa93d595dcaf9864f"},
            "mainnet":{"OMM/sICX": "cx687fa54d7b8328fc560dc7a68a0d44d7f1091a25",
                       "OMM/USDS": "cxa7c9f0afe73e541aee7030a931aec41b81819e74",
                       "OMM/IUSDC": "cx53f4dd2c3243f646b3565ecdab6f2c3b7aa6defb"}}
pool_ids = {"sejong":{"OMM/sICX":'0x2c' ,
                     "OMM/USDS":'0x2a' ,
                     "OMM/IUSDC":'0x2b' },
            "mainnet":{"OMM/sICX": 7,
                       "OMM/USDS":8 ,
                       "OMM/IUSDC":6 }}
excluded_assets = {"sejong":{"daoFund":"cxed3684bd6cc4ccde1d7a5eb5a1dbd0cea1980358",
                             "workerToken":"cx291613e09c548a55582da82bc75a8ad450d99f1a",
                             "bOMM": "cxe0359335aaaa625c72d8429805cbf38f0258ee7a"},
                   "mainnet":{"daoFund":"cx48a83a6fbdaa205a060514fd23ad6871c070f896",
                             "workerToken":"cx718eb47bcf1da56a32afa53ad128844adcfc8211",
                             "bOMM": ""},
                             
}

# # set the addresses of the assets here
# assets={"mainnet":{},"sejong":{}}

#set the cutOffTimestamp here
# cutOffTime=1650015900
cutOffTime=1653549300

# set the env
env ="sejong"



In [ ]:
assetMap = { 
    'OMM': 'cx01c67340ca7bb5e8d1a2bddbfdce40d4addbf7f0',
    'bOMM': 'cxe0359335aaaa625c72d8429805cbf38f0258ee7a',
  'dUSDS': 'cx3023cfd7cc407b67fea8de216d409758f5bb10dd',
 'dICX': 'cx3819c716000b7f8ca08da7e6f6996824d7f8bcaa',
 'dIUSDC': 'cx93239368e19d2028b2abca0472772157d44f2b74',
 'dBALN': 'cxeb14f67eeb6742c7a1ff474308ce82d874469703',
 'dbnUSD': 'cx9175856e735c932b120ad3bdc32bb944222d6a35',
 'dOMM': 'cxd1481b55fc35ce145f8a3a8f1508755ec81bebbd',
 'oUSDS': 'cx1b0a15ac99760720d2cdab4d2309801900ddaece',
 'oICX': 'cx400962267deaaa5cc528297657d4c2d1452f9902',
 'oIUSDC': 'cxc9eb72b8014fd4b232bd63a03f3788f4ba753516',
 'oBALN': 'cx6cbfd9197a9981114b8ae5db47ddb584986112d6',
 'obnUSD': 'cx0a9d4b4ac42eb52d6291e40856e05cf346ea7891',
 'oOMM': 'cx09c1b5462ecccf001a7e79bdf0edba7256a20170',
 'OMM/sICX': 'cxf31d5a3077dfa9b94c7385e17ec747e8809cc53f',
 'OMM/USDS': 'cxa7c9f0afe73e541aee7030a931aec41b81819e74',
 'OMM/IUSDC': 'cxbdfea4b4d291c3e01ed8548aa93d595dcaf9864f'
    
}

asset_map = {}
for k,v in assetMap.items():
    asset_map[v] = k

#### Initializing variables

In [ ]:
network = connections[env]
icon_service = IconService(HTTPProvider(network["iconservice"],3))
lpAddress = lps_list[env]
poolIds = pool_ids[env]
excludedAssets = excluded_assets[env]



#### Importing Wallet

In [ ]:
private="fa627ae3c77a5d4a0fa4fdb6d12bbf413cae8da5b756f9376fe1bebce1580aae"
wallet = KeyWallet.load(bytes.fromhex(private))
wallet.get_address()

#### Helper Methods

In [ ]:
def readOnlyCall(_to, _method, _params):
    """
    send transaction to _to
    :param _to: destination conract name
    :param _method: method
    :param _params:  parameters
    :param _value:  value
    :param _wallet: transaction wallet
    :return: status of transaction
    """
    if _to in contracts[env]:
        to = contracts[env][_to]
    else:
        to = _to
    _call = CallBuilder() \
        .from_(wallet.get_address()) \
        .to(to) \
        .method(_method) \
        .params(_params) \
        .build()
    result = icon_service.call(_call)
    return result
    
def hexToInt(value):
    if type(value) is str:
        return int(value,0)
    if type(value) is dict:
        res ={}
        for k,v in value.items():
            if(not v.startswith("cx")):
                res[k]=int(v,0)
        return res

def hexToInt2D(value):
    res = {
        "OMM":{},
        "OMM/IUSDC":{},
        "OMM/USDS":{},
        "OMM/sICX":{},
        "dBALN":{},
        "dICX":{},
        "dIUSDC":{},
        "dOMM":{},
        "dUSDS":{},
        "dbnUSD":{},
        "oBALN":{},
        "oICX":{},
        "oIUSDC":{},
        "oOMM":{},
        "oUSDS":{},
        "obnUSD":{}
            }
    
    for k,v in value.items():  
        for key, value in v.items():
            res[k][key] = int(value, 0)
        
    return res


#### Setting variables

In [ ]:
assets = readOnlyCall("rewardDistribution","getAssets",{})

#### Methods to fetch and calculate data

In [ ]:
def getAssetIndexes():
    """
    asset index and last updated timestamp
    """
    assetIndexes = readOnlyCall("rewardDistribution","getAllAssetLegacyIndexes",{})
    return hexToInt2D(assetIndexes)

def getUserIndexes(user):
    """
    Returns index and rewards asset wise
    """
    userIndexes =readOnlyCall("rewardDistribution","getUserAllLegacyIndexes",{"_user":user})
#     return userIndexes
    response = hexToInt2D(userIndexes)
    return response

def getAssetEmissions():
    emission=readOnlyCall("rewardWeightController","getEmissionRate",{})
    response =hexToInt(emission)
    response[contracts[env]['ommToken']]= response[excludedAssets["bOMM"]]
    return response

def getTotalSupplyOfAssetsInExa():
    supply ={}
    for k,v in lpAddress.items():
        poolId = poolIds[k]
        totalSupply = hexToInt(readOnlyCall("stakedLp","getTotalStaked",{"_id":poolId}))
        totalSupplyExa = convertToExa(totalSupply["totalStaked"],totalSupply["decimals"])
        supply[v]=totalSupplyExa
        
    for asset in assets:
        if asset not in lpAddress.values() and asset not in excludedAssets.values():
            totalSupply=hexToInt(readOnlyCall(asset,"getPrincipalSupply",{'_user': wallet.get_address()}))
            totalSupplyExa = convertToExa(totalSupply["principalTotalSupply"],totalSupply["decimals"])
            supply[asset]=totalSupplyExa
            
    ommTokenSupply = hexToInt(readOnlyCall("ommToken","getTotalStaked",{}))
    supply[contracts[env]["ommToken"]]=convertToExa(ommTokenSupply["totalStaked"],ommTokenSupply["decimals"])
    return supply

def getUserTotalSupplyExa(user):
    supply ={}
    for k,v in lpAddress.items():
        poolId = poolIds[k]
        totalSupply = hexToInt(readOnlyCall("stakedLp","getLPStakedSupply",{"_id":poolId,"_user":user}))
        totalSupplyExa = convertToExa(totalSupply["principalUserBalance"],totalSupply["decimals"])
        supply[v]=totalSupplyExa
        
    for asset in assets:
        if asset not in lpAddress.values() and asset not in excludedAssets.values():
            totalSupply=hexToInt(readOnlyCall(asset,"getPrincipalSupply",{"_user":user}))
            totalSupplyExa = convertToExa(totalSupply["principalUserBalance"],totalSupply["decimals"])
            supply[asset]=totalSupplyExa
    ommTokenSupply = hexToInt(readOnlyCall("ommToken","getPrincipalSupply",{"_user":user}))
    supply[contracts[env]["ommToken"]]=convertToExa(ommTokenSupply["principalUserBalance"],ommTokenSupply["decimals"])
    return supply

def calculateIndexesAtCutOffTime(initialAssetIndexes, total, emission):  
    result = {}
    for k,v in initialAssetIndexes.items():
        timestamp = v['lastUpdateTimestamp']
        if total[assetMap[k]]>0:
            var = exaDiv((cutOffTime-timestamp)*emission[assetMap[k]],total[assetMap[k]])
            newIndex = var+v['index']
        else:
            newIndex=0

        result[k]=newIndex
    return result

def getUserWorkingBalances(user):
    balances ={}
    for k,v in lpAddress.items():
        poolId = poolIds[k]
        params ={"_id":poolId,"userAddr":user,"assetAddr":v}
        wBalance = hexToInt(readOnlyCall("rewardDistribution","getUserBalance",params))
        balances[v]=wBalance["userBalance"]
    for asset in assets:
        if asset not in lpAddress.values() and asset not in excludedAssets.values():
            poolId = 0
            params ={"_id":poolId,"userAddr":user,"assetAddr":asset}
            wBalance=hexToInt(readOnlyCall("rewardDistribution","getUserBalance",params))
            balances[asset]=wBalance["userBalance"]
    return balances

#### Getting initial assets data

In [ ]:
initialAssetIndexes = getAssetIndexes() # index and timestamp
initialAssetTotalBalancesExa = getTotalSupplyOfAssetsInExa()
assetEmissions =getAssetEmissions()

#### Getting and saving the initial users asset specific data

In [ ]:
completedUsers = []
def initializeCompletedUsers():
    comp={}

    for user in users_list:
        comp[user]=False
    return comp

In [ ]:
users_list=getUsersList()
completedUsers=initializeCompletedUsers()
userAllData =[]


In [ ]:
expectedAssetIndexes = calculateIndexesAtCutOffTime(initialAssetIndexes, initialAssetTotalBalancesExa, assetEmissions)

In [ ]:
def fetch_(user):
    if not completedUsers[user]:
        userIndex = getUserIndexes(user)
        userBalancesExa = getUserTotalSupplyExa(user)
        for k,v in userIndex.items():
            data = {}
            additionalUnclaimedReward = exaMul(expectedAssetIndexes[k]-v['index'],userBalancesExa[assetMap[k]])
            data["user"]=user
            data["asset"]=k
            data["oldIndex"]=v['index']
            data["userSupply"]=userBalancesExa[assetMap[k]]
            data["expectedNewIndex"]=expectedAssetIndexes[k]
            data["expectedUnclaimedReward"]=userIndex[k]['reward']+additionalUnclaimedReward
            data["actualUnclaimedReward"] = userIndex[k]['reward']
            userAllData.append(data)
        completedUsers[user]=True
    else:
        print(user)

In [ ]:
# BLOCK A
a = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor(max_workers=32) as executor:
    executor.map(fetch_, users_list) # addr

b = time.perf_counter()
b-a

In [ ]:
len(users_list) == len(userAllData)/16

In [ ]:
# Check if all user data is fetched
# if any left, call block A again
print("remaining users:>")
for k,v in completedUsers.items():
    if not v:
        print(k)

In [ ]:
# Dump the user data to a csv
csvColumns =["user","asset","oldIndex","userSupply","expectedNewIndex","expectedUnclaimedReward", "actualUnclaimedReward"]
csvFile = "UsersData1.csv"
try:
    with open(csvFile,'w') as csvFile:
        writer= csv.DictWriter(csvFile,fieldnames=csvColumns)
        writer.writeheader()
        for data in userAllData:
            writer.writerow(data)
except:
    print("I/O error")


## MIGRATION

In [ ]:
# Verify working balance and working total


In [ ]:
completedUsers = []
def initializeCompletedUsers():
    comp={}

    for user in users_list:
        comp[user]=False
    return comp

In [ ]:
users_list=getUsersList()
completedUsers=initializeCompletedUsers()
userAllData =[]

In [ ]:
expectedAssetIndexes = calculateIndexesAtCutOffTime(initialAssetIndexes, initialAssetTotalBalancesExa, assetEmissions)

In [ ]:
def fetch_(user):
    if not completedUsers[user]:
        userIndex = getUserIndexes(user)
        userBalancesExa = getUserTotalSupplyExa(user)
        for k,v in userIndex.items():
            data = {}
            additionalUnclaimedReward = exaMul(expectedAssetIndexes[k]-v['index'],userBalancesExa[assetMap[k]])
            data["user"]=user
            data["asset"]=k
            data["oldIndex"]=v['index']
            data["userSupply"]=userBalancesExa[assetMap[k]]
            data["expectedNewIndex"]=expectedAssetIndexes[k]
            data["expectedUnclaimedReward"]=userIndex[k]['reward']+additionalUnclaimedReward
            data["actualUnclaimedReward"] = userIndex[k]['reward']
            userAllData.append(data)
        completedUsers[user]=True
    else:
        print(user)

In [ ]:
# BLOCK B
a = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor(max_workers=32) as executor:
    executor.map(fetch_, users_list) # addr

b = time.perf_counter()
b-a

In [ ]:
len(users_list) == len(userAllData)/16

In [ ]:
# Check if all user data is fetched
# if any left, call block A again
print("remaining users:>")
for k,v in completedUsers.items():
    if not v:
        print(k)

In [ ]:
# Dump the user data to a csv
csvColumns =["user","asset","oldIndex","userSupply","expectedNewIndex","expectedUnclaimedReward", "actualUnclaimedReward"]
csvFile = "UsersDataAfter.csv"
try:
    with open(csvFile,'w') as csvFile:
        writer= csv.DictWriter(csvFile,fieldnames=csvColumns)
        writer.writeheader()
        for data in userAllData:
            writer.writerow(data)
except:
    print("I/O error")


In [ ]:
# Open these 2 CSV files on Excel
# Sort both these files based on address and asset
# Copy oldIndex and actualUnclaimedReward from  to UsersDataBefore
# Verify UsersDataBefore["expectedNewIndex"] == UsersDataAfter["oldIndex"]
# Verify UsersDataBefore["expectedUnclaimedReward"] == UsersDataAfter["actualUnclaimedReward"]